# Pytorch for Boston Classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import (
    Dataset, DataLoader
)

from sklearn.model_selection import train_test_split

from eleven.random import random_seeds
from eleven.data import create_boston_classification

In [2]:
# Some convenient formatting
torch.set_printoptions(precision=2, sci_mode=True)

In [3]:
device = torch.device('cpu')

In [4]:
x, y = create_boston_classification()

In [5]:
x = torch.tensor(x.values, dtype=torch.float32)
y = torch.tensor(y.values)

In [6]:
# Scikit-learn can handle any array like objects, including torch tensors
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42
)

## Creating  a Pytorch Dataset

In [7]:
class Boston(Dataset):
    """Boston Classification
    
    Pytorch datasets require two methods:
    `__len__`, returns the number of samples 
    in the dataset, and `__getitem__`, which
    returns the next sample of data given by an 
    index `idx`.
    
    Args:
        x: features
        y: labels
    """
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Boston(x={self.x.shape}, y={self.y.shape})"
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [8]:
train_data = Boston(x_train, y_train)
valid_data = Boston(x_test, y_test)

In [9]:
print(f"Train Dataset: {train_data}")
print(f"Valid Dataset: {valid_data}")

Train Dataset: Boston(x=torch.Size([339, 13]), y=torch.Size([339]))
Valid Dataset: Boston(x=torch.Size([167, 13]), y=torch.Size([167]))


## A Linear Model, Multi-Class Logistic Regression

In [10]:
class LinearModel(nn.Module):
    
    def __init__(self, input_dim, num_classes):
        super(LinearModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x

In [11]:
# Our features have 13 dimensions. Our model will 
# learn weigths for each of those features. 
model = LinearModel(13, 3)

## DataLoaders

In [12]:
train_loader = DataLoader(train_data, batch_size=1)
valid_loader = DataLoader(valid_data, batch_size=1)

### DataLoader Usage:

We iterate over the dataloader, getting batches of size `batch_size`, and hand that to our model, getting softmax scores of size `(batch_size, num_classes)`.

In [13]:
for idx, (data, target) in enumerate(train_loader):
    softmax_scores = model(data)
    print(f"Model Output Shape: {softmax_scores.shape}")
    if idx == 0:
        break

Model Output Shape: torch.Size([1, 3])


### Repeatable Minibatches

If you would like have reproducible minibatches, you can use the `random_seeds` context manager I added in `eleven.random`:

In [14]:
with random_seeds():
    for idx, (data, target) in enumerate(train_loader):
        print(f"Softmax Scores: {model(data)}")
        if idx == 4:
            break

Softmax Scores: tensor([[0.00e+00, -4.29e+01, -3.03e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -5.97e+01, -2.22e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -6.06e+01, -2.35e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -3.96e+01, -3.24e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -6.09e+01, -2.46e+02]], grad_fn=<LogSoftmaxBackward>)


## Training Loop

In [15]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [16]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:
train_data = Boston(x_train, y_train)
valid_data = Boston(x_test, y_test)

train_loader = DataLoader(train_data, batch_size=32)
valid_loader = DataLoader(valid_data, batch_size=1)

linear_model = LinearModel(13, 3)
optimizer = optim.Adam(linear_model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)


with random_seeds():
    for epoch in range(500):
        train(linear_model, device, train_loader, optimizer, epoch)
        test(linear_model, device, valid_loader)

Train Epoch: 0 [0/339 (0%)]	Loss: 88.109276
Test set: Average loss: 64.5118, Accuracy: 50/167 (30%)

Train Epoch: 1 [0/339 (0%)]	Loss: 78.371849
Test set: Average loss: 59.2893, Accuracy: 46/167 (28%)

Train Epoch: 2 [0/339 (0%)]	Loss: 70.839890
Test set: Average loss: 54.4152, Accuracy: 47/167 (28%)

Train Epoch: 3 [0/339 (0%)]	Loss: 64.809029
Test set: Average loss: 49.7297, Accuracy: 48/167 (29%)

Train Epoch: 4 [0/339 (0%)]	Loss: 59.432533
Test set: Average loss: 45.0785, Accuracy: 51/167 (31%)

Train Epoch: 5 [0/339 (0%)]	Loss: 54.107738
Test set: Average loss: 40.3824, Accuracy: 51/167 (31%)

Train Epoch: 6 [0/339 (0%)]	Loss: 48.650223
Test set: Average loss: 35.6601, Accuracy: 51/167 (31%)

Train Epoch: 7 [0/339 (0%)]	Loss: 43.117374
Test set: Average loss: 30.9626, Accuracy: 54/167 (32%)

Train Epoch: 8 [0/339 (0%)]	Loss: 37.609028
Test set: Average loss: 26.3194, Accuracy: 56/167 (34%)

Train Epoch: 9 [0/339 (0%)]	Loss: 32.189156
Test set: Average loss: 21.7277, Accuracy: 60/1

Test set: Average loss: 1.0277, Accuracy: 106/167 (63%)

Train Epoch: 83 [0/339 (0%)]	Loss: 1.017320
Test set: Average loss: 1.0123, Accuracy: 106/167 (63%)

Train Epoch: 84 [0/339 (0%)]	Loss: 1.000740
Test set: Average loss: 0.9974, Accuracy: 106/167 (63%)

Train Epoch: 85 [0/339 (0%)]	Loss: 0.984499
Test set: Average loss: 0.9830, Accuracy: 106/167 (63%)

Train Epoch: 86 [0/339 (0%)]	Loss: 0.968599
Test set: Average loss: 0.9689, Accuracy: 106/167 (63%)

Train Epoch: 87 [0/339 (0%)]	Loss: 0.953039
Test set: Average loss: 0.9552, Accuracy: 106/167 (63%)

Train Epoch: 88 [0/339 (0%)]	Loss: 0.937827
Test set: Average loss: 0.9420, Accuracy: 107/167 (64%)

Train Epoch: 89 [0/339 (0%)]	Loss: 0.922962
Test set: Average loss: 0.9291, Accuracy: 107/167 (64%)

Train Epoch: 90 [0/339 (0%)]	Loss: 0.908448
Test set: Average loss: 0.9167, Accuracy: 107/167 (64%)

Train Epoch: 91 [0/339 (0%)]	Loss: 0.894289
Test set: Average loss: 0.9047, Accuracy: 108/167 (65%)

Train Epoch: 92 [0/339 (0%)]	Loss:

Test set: Average loss: 0.6734, Accuracy: 114/167 (68%)

Train Epoch: 167 [0/339 (0%)]	Loss: 0.516655
Test set: Average loss: 0.6729, Accuracy: 114/167 (68%)

Train Epoch: 168 [0/339 (0%)]	Loss: 0.515612
Test set: Average loss: 0.6724, Accuracy: 114/167 (68%)

Train Epoch: 169 [0/339 (0%)]	Loss: 0.514596
Test set: Average loss: 0.6720, Accuracy: 116/167 (69%)

Train Epoch: 170 [0/339 (0%)]	Loss: 0.513608
Test set: Average loss: 0.6715, Accuracy: 116/167 (69%)

Train Epoch: 171 [0/339 (0%)]	Loss: 0.512645
Test set: Average loss: 0.6711, Accuracy: 116/167 (69%)

Train Epoch: 172 [0/339 (0%)]	Loss: 0.511708
Test set: Average loss: 0.6706, Accuracy: 116/167 (69%)

Train Epoch: 173 [0/339 (0%)]	Loss: 0.510795
Test set: Average loss: 0.6702, Accuracy: 116/167 (69%)

Train Epoch: 174 [0/339 (0%)]	Loss: 0.509905
Test set: Average loss: 0.6697, Accuracy: 116/167 (69%)

Train Epoch: 175 [0/339 (0%)]	Loss: 0.509036
Test set: Average loss: 0.6693, Accuracy: 116/167 (69%)

Train Epoch: 176 [0/339 (

Train Epoch: 248 [0/339 (0%)]	Loss: 0.472338
Test set: Average loss: 0.6506, Accuracy: 120/167 (72%)

Train Epoch: 249 [0/339 (0%)]	Loss: 0.471980
Test set: Average loss: 0.6505, Accuracy: 120/167 (72%)

Train Epoch: 250 [0/339 (0%)]	Loss: 0.471624
Test set: Average loss: 0.6504, Accuracy: 120/167 (72%)

Train Epoch: 251 [0/339 (0%)]	Loss: 0.471270
Test set: Average loss: 0.6502, Accuracy: 120/167 (72%)

Train Epoch: 252 [0/339 (0%)]	Loss: 0.470916
Test set: Average loss: 0.6501, Accuracy: 120/167 (72%)

Train Epoch: 253 [0/339 (0%)]	Loss: 0.470565
Test set: Average loss: 0.6500, Accuracy: 120/167 (72%)

Train Epoch: 254 [0/339 (0%)]	Loss: 0.470215
Test set: Average loss: 0.6498, Accuracy: 120/167 (72%)

Train Epoch: 255 [0/339 (0%)]	Loss: 0.469866
Test set: Average loss: 0.6497, Accuracy: 120/167 (72%)

Train Epoch: 256 [0/339 (0%)]	Loss: 0.469520
Test set: Average loss: 0.6496, Accuracy: 120/167 (72%)

Train Epoch: 257 [0/339 (0%)]	Loss: 0.469174
Test set: Average loss: 0.6495, Accur

Test set: Average loss: 0.6434, Accuracy: 125/167 (75%)

Train Epoch: 333 [0/339 (0%)]	Loss: 0.446974
Test set: Average loss: 0.6433, Accuracy: 125/167 (75%)

Train Epoch: 334 [0/339 (0%)]	Loss: 0.446729
Test set: Average loss: 0.6433, Accuracy: 125/167 (75%)

Train Epoch: 335 [0/339 (0%)]	Loss: 0.446488
Test set: Average loss: 0.6432, Accuracy: 125/167 (75%)

Train Epoch: 336 [0/339 (0%)]	Loss: 0.446247
Test set: Average loss: 0.6432, Accuracy: 125/167 (75%)

Train Epoch: 337 [0/339 (0%)]	Loss: 0.446007
Test set: Average loss: 0.6431, Accuracy: 125/167 (75%)

Train Epoch: 338 [0/339 (0%)]	Loss: 0.445769
Test set: Average loss: 0.6431, Accuracy: 125/167 (75%)

Train Epoch: 339 [0/339 (0%)]	Loss: 0.445530
Test set: Average loss: 0.6430, Accuracy: 126/167 (75%)

Train Epoch: 340 [0/339 (0%)]	Loss: 0.445294
Test set: Average loss: 0.6430, Accuracy: 126/167 (75%)

Train Epoch: 341 [0/339 (0%)]	Loss: 0.445059
Test set: Average loss: 0.6429, Accuracy: 126/167 (75%)

Train Epoch: 342 [0/339 (

Test set: Average loss: 0.6394, Accuracy: 128/167 (77%)

Train Epoch: 414 [0/339 (0%)]	Loss: 0.430490
Test set: Average loss: 0.6394, Accuracy: 128/167 (77%)

Train Epoch: 415 [0/339 (0%)]	Loss: 0.430322
Test set: Average loss: 0.6393, Accuracy: 128/167 (77%)

Train Epoch: 416 [0/339 (0%)]	Loss: 0.430154
Test set: Average loss: 0.6393, Accuracy: 128/167 (77%)

Train Epoch: 417 [0/339 (0%)]	Loss: 0.429988
Test set: Average loss: 0.6392, Accuracy: 128/167 (77%)

Train Epoch: 418 [0/339 (0%)]	Loss: 0.429822
Test set: Average loss: 0.6392, Accuracy: 128/167 (77%)

Train Epoch: 419 [0/339 (0%)]	Loss: 0.429657
Test set: Average loss: 0.6391, Accuracy: 128/167 (77%)

Train Epoch: 420 [0/339 (0%)]	Loss: 0.429492
Test set: Average loss: 0.6391, Accuracy: 128/167 (77%)

Train Epoch: 421 [0/339 (0%)]	Loss: 0.429328
Test set: Average loss: 0.6391, Accuracy: 128/167 (77%)

Train Epoch: 422 [0/339 (0%)]	Loss: 0.429164
Test set: Average loss: 0.6390, Accuracy: 128/167 (77%)

Train Epoch: 423 [0/339 (

Test set: Average loss: 0.6359, Accuracy: 130/167 (78%)

Train Epoch: 496 [0/339 (0%)]	Loss: 0.418803
Test set: Average loss: 0.6358, Accuracy: 130/167 (78%)

Train Epoch: 497 [0/339 (0%)]	Loss: 0.418683
Test set: Average loss: 0.6358, Accuracy: 130/167 (78%)

Train Epoch: 498 [0/339 (0%)]	Loss: 0.418564
Test set: Average loss: 0.6357, Accuracy: 130/167 (78%)

Train Epoch: 499 [0/339 (0%)]	Loss: 0.418445
Test set: Average loss: 0.6357, Accuracy: 130/167 (78%)



In [18]:
class MLP(nn.Module):
    
    def __init__(self, input_dim, hid_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hid_dim)
        self.fc2 = nn.Linear(hid_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [19]:
mlp = MLP(13, 10, 3)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)


with random_seeds():
    for epoch in range(500):
        train(mlp, device, train_loader, optimizer, epoch)
        test(mlp, device, valid_loader)

Train Epoch: 0 [0/339 (0%)]	Loss: 56.739006
Test set: Average loss: 34.0999, Accuracy: 64/167 (38%)

Train Epoch: 1 [0/339 (0%)]	Loss: 38.917759
Test set: Average loss: 21.4798, Accuracy: 57/167 (34%)

Train Epoch: 2 [0/339 (0%)]	Loss: 24.334997
Test set: Average loss: 16.1400, Accuracy: 55/167 (33%)

Train Epoch: 3 [0/339 (0%)]	Loss: 18.026031
Test set: Average loss: 11.1404, Accuracy: 58/167 (35%)

Train Epoch: 4 [0/339 (0%)]	Loss: 12.500512
Test set: Average loss: 5.7628, Accuracy: 53/167 (32%)

Train Epoch: 5 [0/339 (0%)]	Loss: 5.544951
Test set: Average loss: 4.2681, Accuracy: 35/167 (21%)

Train Epoch: 6 [0/339 (0%)]	Loss: 3.533892
Test set: Average loss: 2.7534, Accuracy: 66/167 (40%)

Train Epoch: 7 [0/339 (0%)]	Loss: 2.502891
Test set: Average loss: 1.8204, Accuracy: 78/167 (47%)

Train Epoch: 8 [0/339 (0%)]	Loss: 1.635023
Test set: Average loss: 1.4676, Accuracy: 86/167 (51%)

Train Epoch: 9 [0/339 (0%)]	Loss: 1.429811
Test set: Average loss: 1.1819, Accuracy: 86/167 (51%)

T

Test set: Average loss: 0.6685, Accuracy: 120/167 (72%)

Train Epoch: 84 [0/339 (0%)]	Loss: 0.516654
Test set: Average loss: 0.6680, Accuracy: 120/167 (72%)

Train Epoch: 85 [0/339 (0%)]	Loss: 0.515484
Test set: Average loss: 0.6676, Accuracy: 120/167 (72%)

Train Epoch: 86 [0/339 (0%)]	Loss: 0.514404
Test set: Average loss: 0.6672, Accuracy: 120/167 (72%)

Train Epoch: 87 [0/339 (0%)]	Loss: 0.513379
Test set: Average loss: 0.6668, Accuracy: 120/167 (72%)

Train Epoch: 88 [0/339 (0%)]	Loss: 0.512337
Test set: Average loss: 0.6664, Accuracy: 119/167 (71%)

Train Epoch: 89 [0/339 (0%)]	Loss: 0.511312
Test set: Average loss: 0.6660, Accuracy: 120/167 (72%)

Train Epoch: 90 [0/339 (0%)]	Loss: 0.510340
Test set: Average loss: 0.6656, Accuracy: 121/167 (72%)

Train Epoch: 91 [0/339 (0%)]	Loss: 0.509367
Test set: Average loss: 0.6652, Accuracy: 122/167 (73%)

Train Epoch: 92 [0/339 (0%)]	Loss: 0.508508
Test set: Average loss: 0.6648, Accuracy: 121/167 (72%)

Train Epoch: 93 [0/339 (0%)]	Loss:

Test set: Average loss: 0.6502, Accuracy: 124/167 (74%)

Train Epoch: 166 [0/339 (0%)]	Loss: 0.470315
Test set: Average loss: 0.6501, Accuracy: 124/167 (74%)

Train Epoch: 167 [0/339 (0%)]	Loss: 0.469947
Test set: Average loss: 0.6500, Accuracy: 124/167 (74%)

Train Epoch: 168 [0/339 (0%)]	Loss: 0.469633
Test set: Average loss: 0.6499, Accuracy: 124/167 (74%)

Train Epoch: 169 [0/339 (0%)]	Loss: 0.469395
Test set: Average loss: 0.6498, Accuracy: 124/167 (74%)

Train Epoch: 170 [0/339 (0%)]	Loss: 0.469147
Test set: Average loss: 0.6496, Accuracy: 124/167 (74%)

Train Epoch: 171 [0/339 (0%)]	Loss: 0.468880
Test set: Average loss: 0.6495, Accuracy: 123/167 (74%)

Train Epoch: 172 [0/339 (0%)]	Loss: 0.468619
Test set: Average loss: 0.6494, Accuracy: 123/167 (74%)

Train Epoch: 173 [0/339 (0%)]	Loss: 0.468511
Test set: Average loss: 0.6492, Accuracy: 123/167 (74%)

Train Epoch: 174 [0/339 (0%)]	Loss: 0.468166
Test set: Average loss: 0.6491, Accuracy: 123/167 (74%)

Train Epoch: 175 [0/339 (

Test set: Average loss: 0.6475, Accuracy: 128/167 (77%)

Train Epoch: 250 [0/339 (0%)]	Loss: 0.456441
Test set: Average loss: 0.6473, Accuracy: 128/167 (77%)

Train Epoch: 251 [0/339 (0%)]	Loss: 0.456261
Test set: Average loss: 0.6472, Accuracy: 128/167 (77%)

Train Epoch: 252 [0/339 (0%)]	Loss: 0.456094
Test set: Average loss: 0.6471, Accuracy: 128/167 (77%)

Train Epoch: 253 [0/339 (0%)]	Loss: 0.455938
Test set: Average loss: 0.6470, Accuracy: 128/167 (77%)

Train Epoch: 254 [0/339 (0%)]	Loss: 0.455782
Test set: Average loss: 0.6469, Accuracy: 128/167 (77%)

Train Epoch: 255 [0/339 (0%)]	Loss: 0.455622
Test set: Average loss: 0.6468, Accuracy: 128/167 (77%)

Train Epoch: 256 [0/339 (0%)]	Loss: 0.455462
Test set: Average loss: 0.6467, Accuracy: 128/167 (77%)

Train Epoch: 257 [0/339 (0%)]	Loss: 0.455300
Test set: Average loss: 0.6466, Accuracy: 128/167 (77%)

Train Epoch: 258 [0/339 (0%)]	Loss: 0.455140
Test set: Average loss: 0.6465, Accuracy: 128/167 (77%)

Train Epoch: 259 [0/339 (

Test set: Average loss: 0.6410, Accuracy: 129/167 (77%)

Train Epoch: 334 [0/339 (0%)]	Loss: 0.443317
Test set: Average loss: 0.6409, Accuracy: 129/167 (77%)

Train Epoch: 335 [0/339 (0%)]	Loss: 0.443059
Test set: Average loss: 0.6409, Accuracy: 129/167 (77%)

Train Epoch: 336 [0/339 (0%)]	Loss: 0.443002
Test set: Average loss: 0.6407, Accuracy: 129/167 (77%)

Train Epoch: 337 [0/339 (0%)]	Loss: 0.442796
Test set: Average loss: 0.6408, Accuracy: 129/167 (77%)

Train Epoch: 338 [0/339 (0%)]	Loss: 0.442556
Test set: Average loss: 0.6407, Accuracy: 129/167 (77%)

Train Epoch: 339 [0/339 (0%)]	Loss: 0.442585
Test set: Average loss: 0.6406, Accuracy: 129/167 (77%)

Train Epoch: 340 [0/339 (0%)]	Loss: 0.442528
Test set: Average loss: 0.6404, Accuracy: 129/167 (77%)

Train Epoch: 341 [0/339 (0%)]	Loss: 0.442237
Test set: Average loss: 0.6405, Accuracy: 129/167 (77%)

Train Epoch: 342 [0/339 (0%)]	Loss: 0.441969
Test set: Average loss: 0.6405, Accuracy: 129/167 (77%)

Train Epoch: 343 [0/339 (

Test set: Average loss: 0.6359, Accuracy: 130/167 (78%)

Train Epoch: 415 [0/339 (0%)]	Loss: 0.432094
Test set: Average loss: 0.6359, Accuracy: 130/167 (78%)

Train Epoch: 416 [0/339 (0%)]	Loss: 0.431966
Test set: Average loss: 0.6358, Accuracy: 130/167 (78%)

Train Epoch: 417 [0/339 (0%)]	Loss: 0.431840
Test set: Average loss: 0.6357, Accuracy: 130/167 (78%)

Train Epoch: 418 [0/339 (0%)]	Loss: 0.431710
Test set: Average loss: 0.6357, Accuracy: 130/167 (78%)

Train Epoch: 419 [0/339 (0%)]	Loss: 0.431575
Test set: Average loss: 0.6356, Accuracy: 130/167 (78%)

Train Epoch: 420 [0/339 (0%)]	Loss: 0.431442
Test set: Average loss: 0.6356, Accuracy: 130/167 (78%)

Train Epoch: 421 [0/339 (0%)]	Loss: 0.431310
Test set: Average loss: 0.6355, Accuracy: 130/167 (78%)

Train Epoch: 422 [0/339 (0%)]	Loss: 0.431178
Test set: Average loss: 0.6355, Accuracy: 130/167 (78%)

Train Epoch: 423 [0/339 (0%)]	Loss: 0.431045
Test set: Average loss: 0.6354, Accuracy: 130/167 (78%)

Train Epoch: 424 [0/339 (

Test set: Average loss: 0.6311, Accuracy: 132/167 (79%)

Train Epoch: 496 [0/339 (0%)]	Loss: 0.422051
Test set: Average loss: 0.6310, Accuracy: 132/167 (79%)

Train Epoch: 497 [0/339 (0%)]	Loss: 0.421683
Test set: Average loss: 0.6310, Accuracy: 132/167 (79%)

Train Epoch: 498 [0/339 (0%)]	Loss: 0.421643
Test set: Average loss: 0.6309, Accuracy: 132/167 (79%)

Train Epoch: 499 [0/339 (0%)]	Loss: 0.421395
Test set: Average loss: 0.6309, Accuracy: 132/167 (79%)

